## UN - tried Soap - didn't finish trying

In [2]:
import PyXML 
import SOAPpy

ns = 'http://www.webservicex.net'
url = '%s/WeatherForecast.asmx' % ns

server = SOAPpy.SOAPProxy( url, namespace=ns )

ImportError: No module named 'PyXML'

## Scrapping from UN search website

In [4]:

###### Link to code: http://nbviewer.jupyter.org/github/psychemedia/dataviz4development/blob/master/dataloaders/UNdata%20Informal%20API.ipynb

#Load in some libraries to handle the web page requests and the web page parsing...
import requests
from bs4 import BeautifulSoup
from urllib.parse import parse_qs

#The scraper will be limited to just the first results page...
def searchUNdata(q):
    ''' Run a search on the UN data website and scrape the results '''
    
    params={'q':q}
    url='http://data.un.org/Search.aspx'

    response = requests.get(url,params=params)

    soup=BeautifulSoup(response.content, "lxml")

    results={}

    #Get the list of results
    searchresults=soup.findAll('div',{'class':'Result'})
    
    #For each result, parse out the name of the dataset, the datamart ID and the data filter ID
    for result in searchresults:
        h2=result.find('h2')
        #We can find everything we need in the <a> tag...
        a=h2.find('a')
        p=parse_qs(a.attrs['href'])
        results[a.text]=(p['d'][0],p['f'][0])

    return results



In [14]:
#A couple of helper functions to let us display the results

results=searchUNdata('carbon dioxide')

def printResults(results):
    ''' Nicely print the search results '''
    
    for result in results.keys():
        print(result)


def unDataSearch(q):
    ''' Simple function to take a search phrase, run the search on the UN data site, and print and return the results. '''
    
    results=searchUNdata(q)
    printResults(results)
    return results

printResults(results)

#q='carbon dioxide'
#unDataSearch(q)



Emissions of carbon dioxide
Carbon dioxide emissions (CO2), thousand metric tons of CO2 (CDIAC)
Carbon dioxide emissions (CO2), metric tons of CO2 per capita (CDIAC)
Carbon dioxide (CO2) Emissions without Land Use, Land-Use Change and Forestry (LULUCF), in kilotonne CO2 equivalent
Carbon dioxide emissions (CO2), metric tons of CO2 per capita (UNFCCC)
Carbon dioxide emissions (CO2), kg CO2 per $1 GDP (PPP) (UNFCCC)
Emissions of carbon dioxide per unit of Manufacturing Value Added
Trade of goods , US$, HS 1992, 28 Inorganic chemicals, precious metal compound, isotope
Carbon dioxide emissions (CO2), kg CO2 per $1 GDP (PPP) (CDIAC)
Emissions of carbon dioxide per unit of GDP (PPP)
Carbon dioxide emissions (CO2), thousand metric tons of CO2 (UNFCCC)


In [38]:

from io import BytesIO
import zipfile
import pandas as pd

def getUNdata(undataSearchResults,dataset):
    ''' Download a named dataset from the UN Data website and load it into a pandas dataframe '''
    
    # How to use HyperLink to get dataset? 
    
    datamartID,seriesRowID=undataSearchResults[dataset]
    
    url='http://data.un.org/Handlers/DownloadHandler.ashx?DataFilter='+seriesRowID+'&DataMartId='+datamartID+'&Format=csv'

    r = requests.get(url)
    
    
    s=BytesIO(r.content)
    z = zipfile.ZipFile(s)
    
    #Show the files in the zip file
    #z.namelist()
    
    #Let's assume we just get one file per zip...
    #Drop any all blank columns
    df=pd.read_csv( BytesIO( z.read( z.namelist()[0] ) )).dropna(axis=1,how='all')
    
    #Create a function that automatically drops the footnotes and any empty rows
    #def cleaned_df(df, footnotes=False):
    #    if footnotes:
    #        return df
    #    return df[:pd.isnull(dd).all(1).nonzero()[0][0]].drop(['Value Footnotes','Value Footnotes.1'], 1)

    #df = cleaned_df(df)
    
    return df

dd=getUNdata(results,'Carbon dioxide emissions (CO2), metric tons of CO2 per capita (CDIAC)')

#Preview the last few rows
dd[1:8]

,Country or Area,Year,Value
1,Afghanistan,2010,0.298290
2,Afghanistan,2009,0.244571
3,Afghanistan,2008,0.156001
4,Afghanistan,2007,0.086563
5,Afghanistan,2006,0.064666
6,Afghanistan,2005,0.053838
7,Afghanistan,2004,0.039848


In [22]:
def searchUNdata(q):
    ''' Run a search on the UN data website and scrape the results '''
    
    params={'q':q}
    url='http://data.un.org/Search.aspx'

    response = requests.get(url,params=params)

    soup=BeautifulSoup(response.content, "lxml")
    
    results=soup.get_text()
    print(results)
    
    #Get the list of results
    searchresults=soup.findAll('div',{'class':'Result'})
        
    return searchresults

searchUNdata('carbon dioxide')



	UNdata | search results





















Data
Glossary
Metadata
API
More








About UNdata
Advanced search
Country profiles
Explorer




Results
11 results found for carbon dioxide. (0.45 seconds)



For carbon dioxide                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

[<div class="Result">
 <h2><a href="Data.aspx?q=carbon+dioxide&amp;d=MDG&amp;f=seriesRowID%3a749"><b>Carbon</b> <b>dioxide</b> emissions (CO2), thousand metric tons of CO2 (CDIAC)</a></h2>
 <div>
                         4575 records<br/>
                         Source: <a class="ds" href="javascript:;" onclick="ShowDataMartInfo(this, 'MDG')">Millennium Development Goals Database</a> | <span><a href="http://unstats.un.org/unsd/" style="color: Green;" target="blank;">United Nations Statistics Division</a></span>
 </div>
 <div class="IconBar">
 <span id="ctl00_main_results_rptResults_ctl00_act_preview">
 <a href="javascript:;" onclick="Preview('MDG', 'seriesRowID%3a749')"><img alt="Preview" src="_Images/IconPreview.png" title="Preview a sample of the data table"/></a>
 <a href="javascript:;" onclick="Preview('MDG', 'seriesRowID%3a749')" title="Preview a sample of the data table">Preview</a>
 </span>
 <span id="ctl00_main_results_rptResults_ctl00_act_download">
 <a href="javascript:;" on

## UN DATA - TRYING TO SAVE SEARCH_INDICATOR

In [29]:
import wbdata 
import pickle 

#wbdata.search_indicators("gdp")

#f = open('myfile.txt','wb')
#f.write(wbdata.search_indicators("gdp"))
#close(f)

data = wbdata.search_indicators("gdp")
#indicators[1:3]
df = wbdata.get_dataframe("Tax payments (number)", convert_date=True)
df = df.reset_index()
df[1:5]

#save as txt file
output = open('myfile.txt', 'wb')
pickle.dump(data,output)
output.close()

#read back
newfile= open('myfile.txt', 'rb')
data2 = pickle.load(newfile)
newfile.close()

print(data2)


IndexError: list index out of range

In [ ]:
f = open('myfile.txt','wb')
f.write(wbdata.search_indicators("gdp"))
close(f)

In [6]:
import wbdata
import sys
sys.stdout = open('myfile.txt','wb')
wbdata.search_indicators("gdp")
sys.stdout.close()


TypeError: a bytes-like object is required, not 'str'

In [3]:
data = open('myfile.txt','w')
print >>data, wbdata.search_indicators("gdp")


TypeError: unsupported operand type(s) for >>: 'builtin_function_or_method' and '_io.TextIOWrapper'

## WorldBank getting XML file 

In [104]:
##### Tried converting xml -> pandas dataframe -->>> didn't get indicator id (what we need)

import pandas as pd
import requests
from xml.etree import ElementTree

url = "http://api.worldbank.org/indicators"

response = requests.get(url)

root = ElementTree.fromstring(response.content)
                                                                                                                                                  
type(root)   

def convert(root):
    all_records = []
    headers = []
    for i, child in enumerate(root):
        record = []
        for subchild in child:
            record.append(subchild.text)
            if subchild.tag not in headers:
                headers.append(subchild.tag)
        all_records.append(record)
    return pd.DataFrame(all_records, columns=headers)

df = convert(root)

df[1:10]                                                                                                                                                  

,{http://www.worldbank.org}name,{http://www.worldbank.org}source,{http://www.worldbank.org}sourceNote,{http://www.worldbank.org}sourceOrganization,{http://www.worldbank.org}topics
1,Energy intensity of residential sector (GJ/hou...,Sustainable Energy for All,None,None,None
2,Energy intensity of transportation sector (MJ/...,Sustainable Energy for All,None,None,None
3,Energy intensity of service sector (MJ/2011 US...,Sustainable Energy for All,None,None,None
4,Energy intensity of agricultural sector (MJ/$2...,Sustainable Energy for All,Energy intensity of agricultural sector (MJ/$2...,World Bank and International Energy Agency (IE...,None
5,Energy intensity of industrial sector (MJ/$2005),Sustainable Energy for All,Energy intensity of industrial sector (MJ/$200...,World Bank and International Energy Agency (IE...,None
6,Transmission and distribution losses (%),Sustainable Energy for All,Transmission and distribution losses (%): Tran...,World Bank and International Energy Agency (IE...,None
7,Thermal efficiency (%) in power supply,Sustainable Energy for All,Thermal efficiency (%) in power supply: This ...,World Bank and International Energy Agency (IE...,None
8,Energy savings of primary energy (TJ),Sustainable Energy for All,Energy savings of primary energy (TJ): Energy ...,World Bank and International Energy Agency (IE...,None
9,Access to electricity (% of urban population),Sustainable Energy for All,Access to electricity is the percentage of tot...,World Bank Global Electrification Database 2014,None


## WorldBank tried downloading json file 

In [38]:
######### Tried downloading as a json file instead of xml --.> didn't work no idea why 

import requests 

url = "http://api.worldbank.org/indicators"

r = requests.get(url)
json_data = r.json()


JSONDecodeError: Unexpected UTF-8 BOM (decode using utf-8-sig): line 1 column 1 (char 0)

## WorldBank Data - searching indicators in xml file 

In [ ]:
## indicators.xml already downloaded

from xml.etree import ElementTree

tree = ET.parse('indicators.xml')
root = tree.getroot()

# Problem left: how to search through root - an xml element tree object - for 'indicators' 